In [1]:
import json
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold, SelectPercentile, mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LassoLarsCV
from xgboost import XGBRegressor
from copy import deepcopy
from time import time
from pprint import pprint
import yaml

In [2]:
# Load YAML file
with open("/Users/utkarsh/MMLI/equicat/science/Science_2019_reaction_handles.yml", 'r') as file:
    reaction_handles = yaml.safe_load(file)

# Extract relevant data from YAML
uts_reaction_handles = reaction_handles["Study 1:"]["UTS"]
train_handles = reaction_handles["Study 2:"]["Train"]
unseen_cat_handles = reaction_handles["Study 2:"]["Test: unseen catalysts"]
unseen_subs_handles = reaction_handles["Study 2:"]["Test: unseen substrates"]
unseen_cat_and_subs_handles = reaction_handles["Study 2:"]["Test: unseen subs and catalysts"]

print("Number of train handles:", len(train_handles))
print("Number of unseen catalysts:", len(unseen_cat_handles))
print("Number of unseen substrates:", len(unseen_subs_handles))
print("Number of unseen catalysts and substrates:", len(unseen_cat_and_subs_handles))

Number of train handles: 384
Number of unseen catalysts: 304
Number of unseen substrates: 216
Number of unseen catalysts and substrates: 171


In [3]:
# Function to load embeddings from JSON file and strip family prefix
def load_embeddings(file_path):
    with open(file_path, 'r') as f:
        raw_embeddings = json.load(f)
    
    embeddings = {}
    family_pattern = re.compile(r'^family\d+_')
    for key, value in raw_embeddings.items():
        stripped_key = family_pattern.sub('', key)
        embeddings[stripped_key] = np.array(value)
    
    return embeddings

# Load embeddings
embeddings = load_embeddings('/Users/utkarsh/MMLI/equicat/develop_op/final_molecule_embeddings.json')
print(f"Loaded embeddings for {len(embeddings)} entities")

# Load Y data
Y_df = pd.read_csv('/Users/utkarsh/MMLI/equicat/science/Y_DATA.csv', dtype={
    'catalyst_id': str,
    'imine_id': str,
    'thiol_id': str,
    'product_id': str
})
print(f"Loaded Y data with {len(Y_df)} rows")

Loaded embeddings for 835 entities
Loaded Y data with 1075 rows


In [4]:
# Create X data using embeddings for catalyst, imine, thiol, and product
X_data = []
Y_data = []
valid_reaction_handles = []

for _, row in Y_df.iterrows():
    catalyst_id = row['catalyst_id']
    imine_id = row['imine_id']
    thiol_id = row['thiol_id']
    product_id = row['product_id']
    reaction_handle = row['reaction_handle']
    
    if all(id in embeddings for id in [catalyst_id, imine_id, thiol_id, product_id]):
        combined_embedding = np.concatenate([
            embeddings[catalyst_id],
            embeddings[imine_id],
            embeddings[thiol_id],
            embeddings[product_id]
        ])
        
        X_data.append(combined_embedding)
        Y_data.append(row['selectivity_ddGact_kcal'])
        valid_reaction_handles.append(reaction_handle)
    else:
        missing_ids = [id for id in [catalyst_id, imine_id, thiol_id, product_id] if id not in embeddings]
        print(f"Missing embedding for reaction: {reaction_handle} - Missing IDs: {missing_ids}")

# Convert to DataFrame
X_df = pd.DataFrame(X_data, index=valid_reaction_handles)
Y_series = pd.Series(Y_data, index=valid_reaction_handles)

print(f"Created dataset with {len(X_df)} samples and {X_df.shape[1]} features")

Missing embedding for reaction: 181_i_1_A - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_1_B - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_1_C - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_1_D - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_1_E - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_2_A - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_2_B - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_2_C - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_2_D - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_2_E - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_3_A - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_3_B - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_3_C - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_3_D - Missing IDs: ['181_i']
Missing embedding for reaction: 181_i_3_E - Missing IDs: ['181

In [5]:
# Helper function for plotting
def multiplot_and_print(estimator, X_train, Y_train, comb_partitions, title, verbose=1, file_dpi=800):
    predicted_train = estimator.predict(X_train)
    r2_train = r2_score(Y_train, predicted_train)
    mae_train = mean_absolute_error(Y_train, predicted_train)

    if verbose:
        print(title)
        print(f"Train R^2: {r2_train:0.5f}, train MAE: {mae_train:0.5f}")

    fig, ax = plt.subplots()

    ax.scatter(
        Y_train,
        predicted_train,
        color="gray",
        label=f"Train (r2= {r2_train:0.3f}, MAE={mae_train:0.3f})"
    )

    for part_name, part_data in comb_partitions.items():
        predicted_test = estimator.predict(part_data[0])
        r2_test = r2_score(part_data[1], predicted_test)
        mae_test = mean_absolute_error(part_data[1], predicted_test)

        if verbose:
            print(f"Test R^2: {r2_test:0.5f}, test MAE: {mae_test:0.5f}")

        ax.scatter(
            part_data[1],
            predicted_test,
            color=part_data[2],
            label=f"{part_name} (r2= {r2_test:0.3f}, MAE={mae_test:0.3f})"
        )

    ax.set_title(title)
    ax.set_xlabel("Observed $\Delta \Delta G^\u2021 [\mathrm{kcal\;mol^{-1}}]$")
    ax.set_ylabel("Predicted $\Delta \Delta G^\u2021 [\mathrm{kcal\;mol^{-1}}]$")
    ax.set_ylim(-3, 3)
    ax.set_xlim(-3, 3)

    plt.legend()
    plt.tight_layout()
    plt.savefig(title + ".png", dpi=file_dpi)
    plt.close()

In [7]:
# Prepare train and test data
valid_train_handles = [handle for handle in train_handles if handle in X_df.index]
X_train = X_df.loc[valid_train_handles]
Y_train = Y_df.loc[Y_df['reaction_handle'].isin(valid_train_handles), ["selectivity_ddGact_kcal"]]
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)

comb_partitions = {}

valid_unseen_subs_handles = [handle for handle in unseen_subs_handles if handle in X_df.index]
X_sub_test = X_df.loc[valid_unseen_subs_handles]
Y_sub_test = Y_df.loc[Y_df['reaction_handle'].isin(valid_unseen_subs_handles), ["selectivity_ddGact_kcal"]]
comb_partitions["Unseen substrates"] = (X_sub_test, Y_sub_test, "green")
print("X_sub_test shape:", X_sub_test.shape)
print("Y_sub_test shape:", Y_sub_test.shape)

valid_unseen_cat_handles = [handle for handle in unseen_cat_handles if handle in X_df.index]
X_cat_test = X_df.loc[valid_unseen_cat_handles]
Y_cat_test = Y_df.loc[Y_df['reaction_handle'].isin(valid_unseen_cat_handles), ["selectivity_ddGact_kcal"]]
comb_partitions["Unseen catalysts"] = (X_cat_test, Y_cat_test, "purple")
print("X_cat_test shape:", X_cat_test.shape)
print("Y_cat_test shape:", Y_cat_test.shape)

valid_unseen_cat_and_subs_handles = [handle for handle in unseen_cat_and_subs_handles if handle in X_df.index]
X_subcat_test = X_df.loc[valid_unseen_cat_and_subs_handles]
Y_subcat_test = Y_df.loc[Y_df['reaction_handle'].isin(valid_unseen_cat_and_subs_handles), ["selectivity_ddGact_kcal"]]
comb_partitions["Unseen subs and cats"] = (X_subcat_test, Y_subcat_test, "blue")
print("X_subcat_test shape:", X_subcat_test.shape)
print("Y_subcat_test shape:", Y_subcat_test.shape)

f_select_model = RandomForestRegressor(n_estimators=500, n_jobs=64, random_state=1234)

base_pipeline = Pipeline(steps=[
    ('variance', VarianceThreshold()),
    ('scaler', MinMaxScaler()),
    # ('feature_selection', SelectFromModel(f_select_model, max_features = 30)),
    ('feature_selection', SelectPercentile(mutual_info_regression, percentile=25)),
])

X_train shape: (368, 768)
Y_train shape: (368, 1)
X_sub_test shape: (207, 768)
Y_sub_test shape: (207, 1)
X_cat_test shape: (304, 768)
Y_cat_test shape: (304, 1)
X_subcat_test shape: (171, 768)
Y_subcat_test shape: (171, 1)


In [8]:
# Define models
models = {
    "GBR": GradientBoostingRegressor(n_estimators=500, ccp_alpha=1e-3),
    "SVR": SVR(kernel='poly', degree=3, epsilon=0.05),
    "RF": RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=7),
    "LL": LassoLarsCV(max_iter=5000, cv=5, n_jobs=-1),
    "XGB": XGBRegressor()
}

# XGBoost parameter grid
xgb_param_grid = {
    'model__learning_rate': np.logspace(-4, 0, 20),
    'model__subsample': np.linspace(0.5, 1.0, 10),
    'model__colsample_bytree': np.linspace(0.1, 1.0, 10),
    'model__max_depth': [3, 4, 5, 6, 7, 8, 9, 10, None],
    'model__min_child_weight': np.linspace(1, 10, 10),
    'model__gamma': np.linspace(0, 1, 10),
    'model__n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
}

In [9]:
# Train and evaluate models
for model_name, model in models.items():
    pipe = deepcopy(base_pipeline)
    pipe.steps.append(('model', model))
    
    print(f"\nTraining {model_name}...")
    t0 = time()
    
    if model_name == "XGB":
        search = RandomizedSearchCV(pipe, cv=5, param_distributions=xgb_param_grid,
                                    n_iter=100, n_jobs=64, verbose=3, refit='neg_mean_absolute_error',
                                    scoring=['neg_mean_absolute_error', 'r2'])
        search.fit(X_train, Y_train)
        pipe = search.best_estimator_
        print("Best XGBoost parameters:")
        pprint(search.best_params_)
    else:
        pipe.fit(X_train, Y_train)
    
    print(f"{model_name}: Fitting took {time() - t0:.3f}s.")

    scores = cross_validate(pipe, X_train, Y_train, cv=5, scoring=['neg_mean_absolute_error', 'r2'], return_train_score=True)
    pprint(scores)
    print(f"{model_name} Q_2: {np.mean(scores['test_r2']):.5f}")

    multiplot_and_print(pipe, X_train, Y_train, comb_partitions, f"BPA_Combinatorial_{model_name}")

print("All models trained and evaluated successfully.")


Training GBR...


/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


GBR: Fitting took 2.509s.


/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed wh

{'fit_time': array([1.60480809, 1.46556592, 1.53778934, 1.43741298, 1.64644217]),
 'score_time': array([0.00541997, 0.00436306, 0.00571799, 0.00357604, 0.00379109]),
 'test_neg_mean_absolute_error': array([-0.48911036, -0.57928951, -0.53099207, -0.48269632, -0.36688606]),
 'test_r2': array([-0.11779532, -1.17054749,  0.26641691, -0.12168491, -0.06020306]),
 'train_neg_mean_absolute_error': array([-0.34556285, -0.37381301, -0.32850108, -0.32639431, -0.33683587]),
 'train_r2': array([0.64779298, 0.59758756, 0.60192412, 0.64836692, 0.68800321])}
GBR Q_2: -0.24076
BPA_Combinatorial_GBR
Train R^2: 0.64349, train MAE: 0.34212
Test R^2: 0.69763, test MAE: 0.31181
Test R^2: -0.50550, test MAE: 0.65968
Test R^2: -0.47600, test MAE: 0.66560

Training SVR...


...

[CV 2/5] END model__colsample_bytree=0.5, model__gamma=0.6666666666666666, model__learning_rate=0.3792690190732246, model__max_depth=None, model__min_child_weight=8.0, model__n_estimators=900, model__subsample=1.0; neg_mean_absolute_error: (test=-0.618) r2: (test=-1.456) total time=  18.0s
[CV 1/5] END model__colsample_bytree=0.5, model__gamma=0.6666666666666666, model__learning_rate=0.3792690190732246, model__max_depth=None, model__min_child_weight=8.0, model__n_estimators=900, model__subsample=1.0; neg_mean_absolute_error: (test=-0.573) r2: (test=-0.486) total time=  18.5s
[CV 4/5] END model__colsample_bytree=0.8, model__gamma=0.1111111111111111, model__learning_rate=0.0001, model__max_depth=3, model__min_child_weight=5.0, model__n_estimators=600, model__subsample=0.8888888888888888; neg_mean_absolute_error: (test=-0.625) r2: (test=-0.891) total time=  21.7s
[CV 3/5] END model__colsample_bytree=0.7000000000000001, model__gamma=0.1111111111111111, model__learning_rate=0.23357214690901

/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best XGBoost parameters:
{'model__colsample_bytree': 0.7000000000000001,
 'model__gamma': 0.4444444444444444,
 'model__learning_rate': 0.03359818286283781,
 'model__max_depth': 4,
 'model__min_child_weight': 8.0,
 'model__n_estimators': 800,
 'model__subsample': 0.5555555555555556}
XGB: Fitting took 192.809s.


/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/utkarsh/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expec

{'fit_time': array([1.77148008, 1.51426315, 1.36000896, 2.27825308, 1.60608625]),
 'score_time': array([0.00903201, 0.01003575, 0.00553107, 0.01059294, 0.00563288]),
 'test_neg_mean_absolute_error': array([-0.45103183, -0.51250857, -0.53950462, -0.50193259, -0.36164833]),
 'test_r2': array([ 0.09152317, -0.76095247,  0.19485027, -0.19167602,  0.0377425 ]),
 'train_neg_mean_absolute_error': array([-0.34057979, -0.37001301, -0.32474161, -0.3222842 , -0.33212364]),
 'train_r2': array([0.65245199, 0.60275936, 0.60909897, 0.65365493, 0.69254196])}
XGB Q_2: -0.12570
BPA_Combinatorial_XGB
Train R^2: 0.64879, train MAE: 0.33818
Test R^2: 0.71515, test MAE: 0.30185
Test R^2: -0.04008, test MAE: 0.55536
Test R^2: -0.11822, test MAE: 0.59328
All models trained and evaluated successfully.


In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance

def analyze_feature_importance(model, X, y, feature_names, model_name):
    if hasattr(model, 'feature_importances_'):
        # For tree-based models (RF, GBR, XGB)
        importances = model.feature_importances_
        indices = np.argsort(importances)[::-1]
        
        plt.figure(figsize=(12, 8))
        plt.title(f"Feature Importances ({model_name})")
        plt.bar(range(min(20, len(importances))), importances[indices][:20], align="center")
        plt.xticks(range(min(20, len(importances))), [feature_names[i] for i in indices[:20]], rotation=90)
        plt.tight_layout()
        plt.savefig(f"feature_importances_{model_name.lower()}.png")
        plt.close()
        
        print(f"\nTop 20 features for {model_name}:")
        for f in range(min(20, len(importances))):
            print("%d. %s (%f)" % (f + 1, feature_names[indices[f]], importances[indices[f]]))
    
    # Permutation Importance (works for all models)
    perm_importance = permutation_importance(model, X, y, n_repeats=10, random_state=42, n_jobs=-1)
    
    perm_importances = pd.DataFrame({
        'feature': feature_names[:len(perm_importance.importances_mean)],
        'importance': perm_importance.importances_mean,
        'std': perm_importance.importances_std
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    plt.title(f"Permutation Importances ({model_name})")
    plt.bar(range(20), perm_importances['importance'][:20], yerr=perm_importances['std'][:20], align="center")
    plt.xticks(range(20), perm_importances['feature'][:20], rotation=90)
    plt.tight_layout()
    plt.savefig(f"permutation_importances_{model_name.lower()}.png")
    plt.close()
    
    print(f"\nTop 20 features by permutation importance for {model_name}:")
    print(perm_importances.head(20))